The goal here is to address [#5](https://github.com/LSSTDESC/DC2_visitList/issues/5 ), i.e. if the WFD visits to field 1427 are also being considered uDDF visits.

In [1]:
import numpy as np
import pandas as pd
import os
import time

In [2]:
mainDir = '/global/homes/a/awan/LSST/lsstRepos/DC2_visitList/DC2visitGen/'
nside = 1024
baseDir = '%s/DC2_Run2_Visits_nside%s/'%(mainDir, nside)

Read in the all-band lists for Run 2

In [3]:
outDir = '%s/DC2_Run2_Visits_nside%s/visitLists/'%(mainDir, nside)

In [4]:
data = {}
for survey in ['WFD', 'DD']:
    print('---------------------------------------------------------------------------------------------------')
    print('### Reading in %s files\n'%survey)
    files = [f for f in os.listdir(outDir) if f.__contains__(survey)]
    print('Found %s new files'%len(files))

    if len(files)>1:
        raise ValueError('Somethings wrong: %s'%files)

    print('Reading in %s'%files[0])
    data[survey] = pd.read_csv('%s/%s'%(outDir, files[0]), delimiter=',')

---------------------------------------------------------------------------------------------------
### Reading in WFD files

Found 1 new files
Reading in DC2VisitList_minion1016_v3_WFDvisits_nside1024_WFDvisitsInWFD.csv
---------------------------------------------------------------------------------------------------
### Reading in DD files

Found 1 new files
Reading in DC2VisitList_minion1016_v3_DDvisits_nside1024_DDvisitsInuDD.csv


In [5]:
ind = np.where(data['WFD']['fID']==1427)[0]
print('%s WFD to field 1427 out of %s'%(len(ind), len(data['WFD']['fID'])))

919 WFD to field 1427 out of 50415


Okay so there are WFD visits to the uDDF field ID. Lets see if those visits are in the uDDF list.

In [6]:
i = 0
for ID in data['WFD']['obsHistID'][ind]:
    if ID in data['DD']['obsHistID']: i+=1
        
print('%s of the %s WFD visits to 1427 are in the uDDF list'%(i, len(data['WFD']['fID'])))

0 of the 50415 WFD visits to 1427 are in the uDDF list


Okay great, the WFD visits to field 1427 are *not* in the uDDF list.

Just another quick test: see if there are *any* visits that are common to the two lists

In [7]:
in_both = set(data['WFD']['obsHistID']) - (set(data['WFD']['obsHistID']) - set(data['DD']['obsHistID']) )
print('%s visits in common, based on WFD list'%len(in_both))

in_both = set(data['DD']['obsHistID']) - (set(data['DD']['obsHistID']) - set(data['WFD']['obsHistID']) )
print('%s visits in common, based on uDDF list'%len(in_both))

0 visits in common, based on WFD list
0 visits in common, based on uDDF list


Perfect.

#### We can also quickly check for duplicated visits in the per-band lists

In [8]:
outDir_bands = '%s/DC2_Run2_Visits_nside%s/visitLists/per_band_lists/'%(mainDir, nside)

In [10]:
data = {}
for survey in ['WFD', 'DD']:
    data[survey] = {}
    print('---------------------------------------------------------------------------------------------------')
    print('### Reading in %s files\n'%survey)
    for band in ['u', 'g', 'r', 'i', 'z', 'y']:
        print('## Reading in %s-band files\n'%band)
        # file the filename
        new_files = [f for f in os.listdir(outDir_bands) if f.__contains__('%s-band'%band) and f.__contains__(survey)]
        print('Found %s new files'%len(new_files))
        # there should not be more than 1 file
        if len(new_files)>1:
            raise ValueError('Somethings wrong: %s'%new_files)
        # read in the data
        print('Reading in %s'%new_files[0])
        data[survey][band] = pd.read_csv('%s/%s'%(outDir_bands, new_files[0]), delimiter=' ')

---------------------------------------------------------------------------------------------------
### Reading in WFD files

## Reading in u-band files

Found 1 new files
Reading in DC2_Run2_u-band_WFDvisits.txt
## Reading in g-band files

Found 1 new files
Reading in DC2_Run2_g-band_WFDvisits.txt
## Reading in r-band files

Found 1 new files
Reading in DC2_Run2_r-band_WFDvisits.txt
## Reading in i-band files

Found 1 new files
Reading in DC2_Run2_i-band_WFDvisits.txt
## Reading in z-band files

Found 1 new files
Reading in DC2_Run2_z-band_WFDvisits.txt
## Reading in y-band files

Found 1 new files
Reading in DC2_Run2_y-band_WFDvisits.txt
---------------------------------------------------------------------------------------------------
### Reading in DD files

## Reading in u-band files

Found 1 new files
Reading in DC2_Run2_u-band_uDDFvisits.txt
## Reading in g-band files

Found 1 new files
Reading in DC2_Run2_g-band_uDDFvisits.txt
## Reading in r-band files

Found 1 new files
Readi

Can't check for field specifically since the per-band files dont include the fieldID column. We can only check for the common visits:

In [11]:
for band in data['WFD'].keys():
    print('Band: %s'%band)
    # now check for common visits
    in_both = set(data['WFD'][band]['obsHistID']) - (set(data['WFD'][band]['obsHistID']) - set(data['DD'][band]['obsHistID']) )
    print('%s visits common, based on WFD list'%len(in_both))
    
    in_both = set(data['DD'][band]['obsHistID']) - (set(data['DD'][band]['obsHistID']) - set(data['WFD'][band]['obsHistID']) )
    print('%s visits common, based on DDF list\n'%len(in_both))

Band: u
0 visits common, based on WFD list
0 visits common, based on DDF list

Band: g
0 visits common, based on WFD list
0 visits common, based on DDF list

Band: r
0 visits common, based on WFD list
0 visits common, based on DDF list

Band: i
0 visits common, based on WFD list
0 visits common, based on DDF list

Band: z
0 visits common, based on WFD list
0 visits common, based on DDF list

Band: y
0 visits common, based on WFD list
0 visits common, based on DDF list



Okay great, no visits are in common across any of the per-band lists.